In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.distributions as dist
import numpy as np

In [6]:
# Define the base distribution (a 1D Gaussian)
base_dist = dist.normal.Normal(loc=torch.tensor([0.]), scale=torch.tensor([1.]))

# Define the target distribution (a bimodal distribution)
target_dist = dist.normal.Normal(loc=torch.tensor([-2., 2.]), scale=torch.tensor([0.5, 0.5]))

# Define the dynamics function
class Dynamics(nn.Module):
    def __init__(self):
        super().__init__()

        # Define the layers of the dynamics function
        self.fc1 = nn.Linear(1, 64)
        self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(64, 1)
        self.softplus = nn.Softplus()

    def forward(self, x, t=None):
        """
        This function computes the dynamics of the flow model.

        Args:
            x: the input tensor
            t: the current time (not used in this example)

        Returns:
            The output tensor of the dynamics function and the log determinant of the Jacobian matrix
        """

        # Apply the layers of the dynamics function
        h = torch.relu(self.fc1(x))
        h = torch.relu(self.fc2(h))
        h = self.fc3(h)

        # Compute the scale parameter using the output of the final layer and the softplus activation function
        scale = self.softplus(h)

        # Compute the output of the dynamics function and the log determinant of the Jacobian matrix
        return scale * x, torch.sum(torch.log(scale), dim=-1)

# Define the continuous flow model
class Flow(nn.Module):
    def __init__(self, dynamics):
        super().__init__()

        # Store the dynamics function as a module attribute
        self.dynamics = dynamics

    def forward(self, x, log_jac0=None, reverse=False):
        """
        This function computes the forward or inverse transformation of the flow model.

        Args:
            x: the input tensor
            log_jac0: the initial value of the log determinant of the Jacobian matrix (not used in this example)
            reverse: if True, compute the inverse transformation

        Returns:
            The transformed tensor and the log determinant of the Jacobian matrix
        """

        if not reverse:
            # If computing the forward transformation, apply the dynamics function to the input tensor
            return self.dynamics(x, t=0.)
        else:
            # If computing the inverse transformation, apply the dynamics function to the input tensor in reverse order
            z = x
            log_jac = log_jac0
            for i in range(len(self.dynamics)):
                z, log_det = self.dynamics[-1-i](z, t=0.)
                log_jac = log_jac - log_det
            return z, log_jac

# Define the loss function (negative log-likelihood)
def loss_fn(x, log_jac):
    """
    This function computes the negative log-likelihood of the transformed distribution.

    Args:
        x: the transformed tensor
        log_jac: the log determinant of the Jacobian matrix

    Returns:
        The negative log-likelihood of the transformed distribution
    """

    # Compute the negative log-likelihood of the target distribution using the log probability function of PyTorch's distribution module
    return -target_dist.log_prob(x).sum() - log_jac.sum()

In [7]:
# Define the optimizer (stochastic gradient descent)
dynamics = Dynamics()
flow = Flow([dynamics])
optimizer = optim.SGD(flow.parameters(), lr=0.001)

# Train the flow model
num_epochs = 10000
batch_size = 256
for epoch in range(num_epochs):
    # Generate a batch of samples from the base distribution
    batch_x = base_dist.sample((batch_size,))

    # Transform the samples using the flow model
    y, log_jac = flow(batch_x)

    # Compute the loss
    loss = loss_fn(y, log_jac)

    # Zero the gradients
    optimizer.zero_grad()

    # Compute the gradients
    loss.backward()

    # Update the parameters
    optimizer.step()

    # Print the loss every 10 epochs
    if epoch % 1000 == 0:
        print(f"Epoch {epoch}: Loss = {loss.item()}")

# Evaluate the trained model
# Generate a large number of samples from the transformed distribution
num_samples = 10000
samples = []
for _ in range(num_samples // batch_size):
    # Generate a batch of samples from the base distribution
    batch_x = base_dist.sample((batch_size,))

    # Transform the samples using the flow model
    y, log_jac = flow(batch_x)
    samples.append(y.detach().numpy())

samples = np.concatenate(samples)

# Compute the empirical mean and standard deviation of the transformed samples
mean = np.mean(samples)
std = np.std(samples)

# Print the results
print(f"Transformed distribution")

ValueError: optimizer got an empty parameter list

In [2]:
import torch
import torch.nn as nn
from torch.optim import Adam
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from ffjord.model import FFJORD


# Define a simple 1D Gaussian as the base distribution
class GaussianDistribution(nn.Module):
    def __init__(self, mu, sigma):
        super(GaussianDistribution, self).__init__()
        self.mu = mu
        self.sigma = sigma

    def forward(self, x):
        return (-((x - self.mu)**2) / (2 * self.sigma**2)).exp() / self.sigma


# Define a bimodal distribution as the target distribution
class BimodalDistribution(nn.Module):
    def forward(self, x):
        return torch.exp(-0.5*((x-2)**2)/0.5) + torch.exp(-0.5*((x+2)**2)/0.5)


# Define the time dynamic
class TimeDynamics(nn.Module):
    def __init__(self, dim):
        super(TimeDynamics, self).__init__()
        self.dim = dim

    def forward(self, t, z):
        # Define the time-dependent coefficient function
        # In this example, we use a simple linear function
        a_t = torch.tensor([[1.0 + 0.5 * t]])

        # Multiply the coefficient function with the noise input
        a_t = a_t.expand(z.shape[0], -1)

        # Return the time derivative of z
        return -torch.mm(a_t, z.unsqueeze(-1)).squeeze(-1)


# Define the flow model using FFJORD
class FlowModel(nn.Module):
    def __init__(self, dim, time_dynamics):
        super(FlowModel, self).__init__()
        self.dim = dim

        # Define the FFJORD module with the Gaussian distribution and time dynamics
        self.ffjord = FFJORD(
            dim=self.dim,
            time_dynamics=time_dynamics,
            train_T=True,
            solver='dopri5'
        )

        # Define the base distribution as a 1D Gaussian with mean=0 and standard deviation=1
        self.base_dist = GaussianDistribution(mu=torch.tensor([0.0]), sigma=torch.tensor([1.0]))

    def forward(self, x):
        # Generate samples from the base distribution
        z, log_jac = self.ffjord(x)

        # Compute the log probability density of the target distribution using the change of variable formula
        # log p(x) = log p(z) + log det |dx/dz|
        # We use the log_sum_exp trick to improve numerical stability
        log_p = torch.logsumexp(torch.log(self.base_dist(z)) + log_jac, dim=1)

        # Return the negative log probability as the loss
        return -log_p.mean()


# Define the training loop
def train(model, optimizer, dataloader, epochs):
    for epoch in range(epochs):
        loss_total = 0.0
        for x, _ in dataloader:
            # Reset the gradients
            optimizer.zero_grad()

            # Forward pass
            loss = model(x)

            # Backward pass
            loss.backward()

            # Update parameters
            optimizer.step()

            loss_total += loss.item()

        # Print the average loss over the dataset for each epoch
        print("Epoch {}/{}: Loss = {:.4f}".format(epoch + 1, epochs, loss_total / len(dataloader)))


# Set the random seed for reproducibility
torch.manual_seed(42)

ModuleNotFoundError: No module named 'ffjord.model'; 'ffjord' is not a package

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchdiffeq import odeint_adjoint as odeint

class Dynamics(nn.Module):
    def __init__(self, hidden_size):
        super(Dynamics, self).__init__()
        self.fc1 = nn.Linear(hidden_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, hidden_size)

    def forward(self, t, z):
        h = F.relu(self.fc1(z))
        h = F.relu(self.fc2(h))
        h = self.fc3(h)
        return h

class ContinuousNF(nn.Module):
    def __init__(self, hidden_size, num_blocks, solver):
        super(ContinuousNF, self).__init__()
        self.num_blocks = num_blocks
        self.dynamics = nn.ModuleList([Dynamics(hidden_size) for _ in range(num_blocks)])
        self.solver = solver

    def forward(self, z0):
        zs = [z0]
        log_det_Js = []
        for i in range(self.num_blocks):
            t0 = torch.zeros(z0.size(0)).to(z0.device)
            z1, log_det_J = self.integrate(self.dynamics[i], zs[-1], t0, t0 + 1.0)
            zs.append(z1)
            log_det_Js.append(log_det_J)
        return torch.cat(zs[1:], dim=0), sum(log_det_Js)

    def integrate(self, f, z0, t0, t1):
        def grad(z, t):
            z = torch.autograd.Variable(z, requires_grad=True)
            dzdt = f(t, z)
            d2zdt2 = torch.autograd.grad(dzdt.sum(), z, create_graph=True)[0]
            return dzdt, d2zdt2

        zs = odeint(grad, z0, torch.tensor([t0, t1]).float(), method=self.solver)
        z1, zlog_det_J = zs[-1], 0.0
        for i in range(len(zs) - 1):
            dzdt, d2zdt2 = grad(zs[i], t0 + i * (t1 - t0) / (len(zs) - 1))
            zlog_det_J += torch.log(torch.abs(torch.det(torch.eye(z0.size(-1)).to(z0.device) + (t1 - t0) / (len(zs) - 1) * dzdt + ((t1 - t0) / (len(zs) - 1))**2 * d2zdt2)))
        return z1, zlog_det_J


In [7]:
import torch.optim as optim
from tqdm import tqdm

model = ContinuousNF(5, 3, odeint)
batch_size = 64

# Define optimizer and loss
optimizer = optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.MSELoss()

# Train the model
num_epochs = 1000
for epoch in tqdm(range(num_epochs)):
    # Zero the gradients
    optimizer.zero_grad()

    # Sample a batch of data from the base distribution
    z = torch.randn(batch_size, 1)

    # Compute the transformed samples and log determinant of the Jacobian
    x, log_det = model(z)

    # Compute the negative log-likelihood
    loss = -torch.mean(base_dist.log_prob(z) + log_det)

    # Backpropagate and update parameters
    loss.backward()
    optimizer.step()

    # Print loss every 100 epochs
    if epoch % 100 == 0:
        print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

  0%|                                                  | 0/1000 [00:00<?, ?it/s]


ValueError: only one element tensors can be converted to Python scalars

In [11]:
%pip install ffjord

ERROR: Could not find a version that satisfies the requirement ffjord (from versions: none)
ERROR: No matching distribution found for ffjord
Note: you may need to restart the kernel to use updated packages.


In [27]:
import torch
import torch.nn as nn
from torchdiffeq import odeint_adjoint as odeint

# Define the base distribution (1D Gaussian)
def gaussian(x, mu=0, sigma=1):
    return torch.exp(-0.5 * ((x - mu) / sigma) ** 2) / (sigma * torch.sqrt(2 * torch.tensor(3.141592653589793)))

# Define the target distribution (bimodal)
def bimodal(x):
    return 0.5 * gaussian(x, mu=-2, sigma=0.5) + 0.5 * gaussian(x, mu=2, sigma=1)

# Define the flow model using a neural ODE
class FlowModel(nn.Module):
    def __init__(self):
        super(FlowModel, self).__init__()
        self.fc1 = nn.Linear(1, 64)
        self.fc2 = nn.Linear(64, 1)
        self.t = torch.linspace(0, 1, 10)
        
    def forward(self, t, x):
        h = torch.relu(self.fc1(x))
        h = self.fc2(h)
        return h, torch.zeros_like(h)
    
    def backward(self, z):
        with torch.no_grad():
            x = odeint(self, z, self.t, method='dopri5', rtol=1e-4, atol=1e-5)[-1]
        return x
    
# Define the loss function
def loss_fn(z, log_jacobian):
    return -torch.mean(bimodal(z) + log_jacobian)

# Define the training loop
def train_flow_model(flow_model, optimizer, epochs):
    for epoch in range(epochs):
        optimizer.zero_grad()
        z = torch.randn(128, 1)
        x, log_jacobian = flow_model(0, z) # pass in time t=0
        loss = loss_fn(x, log_jacobian)
        loss.backward()
        optimizer.step()
        if epoch % 100 == 0:
            print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item()}")

# Instantiate the flow model and optimizer
flow_model = FlowModel()
optimizer = torch.optim.Adam(flow_model.parameters(), lr=1e-3)

# Train the flow model
train_flow_model(flow_model, optimizer, epochs=1000)

Epoch [1/1000], Loss: -0.03594920039176941
Epoch [101/1000], Loss: -0.18914613127708435
Epoch [201/1000], Loss: -0.19933584332466125
Epoch [301/1000], Loss: -0.1994125097990036
Epoch [401/1000], Loss: -0.19942568242549896
Epoch [501/1000], Loss: -0.19943687319755554
Epoch [601/1000], Loss: -0.19945496320724487
Epoch [701/1000], Loss: -0.19945508241653442
Epoch [801/1000], Loss: -0.19946414232254028
Epoch [901/1000], Loss: -0.1994565725326538


In [29]:
import torch
from torch import nn
from torchdiffeq import odeint_adjoint as odeint

# Define the continuous-time dynamics using an ordinary differential equation (ODE)
class Dynamics(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(1, 50),
            nn.Tanh(),
            nn.Linear(50, 1),
        )

    def forward(self, t, x):
        # t is time (not used in this case)
        # x is the input
        dxdt = -self.net(x.pow(3))
        return dxdt

# Define the continuous normalizing flow model
class CNF(nn.Module):
    def __init__(self, dynamics):
        super().__init__()
        self.dynamics = dynamics

    def forward(self, x, logpx=None):
        # Reshape x to be a column vector
        x = x.view(-1, 1)

        # Compute the derivative of x with respect to t using the ODE solver
        # We use the adjoint method for backpropagation through the ODE solver
        t = torch.Tensor([0, 1]).to(x)
        xt = odeint(self.dynamics, x, t, rtol=1e-5, atol=1e-6)

        # Reshape xt to be a row vector
        xt = xt.view(1, -1)

        # Compute the log determinant of the Jacobian matrix
        # This is needed for computing the likelihood of the transformed distribution
        logdet = torch.zeros(x.shape[0], device=x.device)

        if logpx is not None:
            logpx = logpx - logdet

        return xt, logpx

# Define the simple 1D Gaussian distribution
class Gaussian:
    def __init__(self, mu, sigma):
        self.mu = mu
        self.sigma = sigma

    def sample(self, n):
        return torch.randn(n) * self.sigma + self.mu

    def log_prob(self, x):
        return -0.5 * ((x - self.mu) / self.sigma) ** 2 - torch.log(torch.sqrt(2 * torch.tensor([3.1415])) * self.sigma)

# Instantiate the continuous normalizing flow model and the simple 1D Gaussian distribution
dynamics = Dynamics()
cnf = CNF(dynamics)
gaussian = Gaussian(mu=0.5, sigma=0.1)

# Sample from the simple 1D Gaussian distribution
x = gaussian.sample(100)

# Transform the samples using the continuous normalizing flow model
xt, logpx = cnf(x)

# Compute the likelihood of the transformed samples under the transformed distribution
log_prob = gaussian.log_prob(xt.view(-1))

# Print the average log likelihood
print(log_prob.mean())

tensor(0.6344, grad_fn=<MeanBackward0>)


In [30]:
import torch
from torchdiffeq import odeint_adjoint as odeint

class NeuralODE(torch.nn.Module):
    def __init__(self):
        super(NeuralODE, self).__init__()
        self.linear1 = torch.nn.Linear(1, 32)
        self.linear2 = torch.nn.Linear(32, 1)
        self.activation = torch.nn.Tanh()
        
    def forward(self, t, x):
        out = self.linear1(x)
        out = self.activation(out)
        out = self.linear2(out)
        return out
    
def bimodal_target(x):
    return torch.exp(-(x-2)**2) + torch.exp(-(x+2)**2)
    
def train_neural_ode(num_epochs=5000):
    # Define the initial condition
    x0 = torch.tensor([0.0], requires_grad=True)
    t = torch.linspace(0.0, 1.0, 100)
    
    # Define the neural ODE
    net = NeuralODE()
    
    # Define the optimizer
    optimizer = torch.optim.Adam(net.parameters(), lr=1e-3)
    
    # Train the neural ODE
    for epoch in range(num_epochs):
        # Forward pass
        pred = odeint(net, x0, t)
        
        # Calculate the loss
        loss = torch.mean((pred[:, 0] - bimodal_target(t))**2)
        
        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # Print the loss every 100 epochs
        if epoch % 100 == 0:
            print("Epoch [{}/{}], Loss: {:.4f}".format(epoch+1, num_epochs, loss.item()))
    
    # Return the final prediction
    return odeint(net, x0, t)

In [32]:
train_neural_ode()